In [1]:
import os

from dotenv import load_dotenv
from pyspark.sql import SparkSession
from delta.pip_utils import configure_spark_with_delta_pip

from engine import ClimaTempoAPI
from etl import cities_etl, forecast_72h_etl

In [2]:
load_dotenv()

builder = SparkSession.builder\
       .appName('raw_etl')\
       .config('spark.sql.warehouse.dir', 'pyspark_tables')\
       .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
       .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
       .config('spark.databricks.delta.retentionDurationCheck.enabled', False) \
       .config('spark.databricks.delta.schema.autoMerge.enabled', True) \
       .config('spark.databricks.delta.checkLatestSchemaOnRead', True) \
       .config('delta.enableChangeDataFeed', True) \
       .config('spark.sql.shuffle.partitions', 10) \
       .config('spark.databricks.preemption.enabled', True) \
       .config('spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite', True)

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()

23/04/25 03:26:02 WARN Utils: Your hostname, spiriel resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface enp3s0)
23/04/25 03:26:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ahow/main_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ahow/.ivy2/cache
The jars for the packages stored in: /home/ahow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8667597a-5204-4b21-a485-52a9a3a46918;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 88ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   | 

23/04/25 03:26:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
TOKEN = os.environ['TOKEN']
DATABASE = os.environ['DATABASE']
CITIES_RAW_TB = os.environ['CITIES_RAW_TB']
FORECAST_72H_RAW_TB = os.environ['FORECAST_72H_RAW_TB']

spark.sql(f"CREATE DATABASE IF NOT EXISTS {DATABASE}").show()

++
||
++
++



In [6]:
api = ClimaTempoAPI(TOKEN)

cities_etl(api, CITIES_RAW_TB, spark, timeout=60, params={'country': 'BR'})
cities_etl(api, CITIES_RAW_TB, spark, timeout=60, params={'country': 'CO'})

cities_df = spark.read.format('delta').table(CITIES_RAW_TB)
cities_df.toPandas()

/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


23/04/25 03:26:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


,id,name,state,country
0,143538,Paraíso das Águas,MS,BR
1,1868,Cachoeira Dourada,MG,BR
2,1899,Canápolis,MG,BR
3,1914,Capinópolis,MG,BR
4,1917,Caldas,MG,BR
...,...,...,...,...
6163,995,Soledad,ND,CO
6164,996,Tumaco,ND,CO
6165,997,Tunja,ND,CO
6166,998,Valledupar,ND,CO


In [7]:
df = forecast_72h_etl(api, FORECAST_72H_RAW_TB, spark, [3477], params={'timeout': 60})

forecast72_df = spark.read.format('delta').table(FORECAST_72H_RAW_TB)
forecast72_df.toPandas()

/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


23/04/25 03:26:26 WARN MemoryManager: Total allocation exceeds 95.00% (981,204,980 bytes) of heap memory
Scaling row group sizes to 91.38% for 8 writers
23/04/25 03:26:26 WARN MemoryManager: Total allocation exceeds 95.00% (981,204,980 bytes) of heap memory
Scaling row group sizes to 81.23% for 9 writers
23/04/25 03:26:26 WARN MemoryManager: Total allocation exceeds 95.00% (981,204,980 bytes) of heap memory
Scaling row group sizes to 73.11% for 10 writers
23/04/25 03:26:26 WARN MemoryManager: Total allocation exceeds 95.00% (981,204,980 bytes) of heap memory
Scaling row group sizes to 66.46% for 11 writers
23/04/25 03:26:26 WARN MemoryManager: Total allocation exceeds 95.00% (981,204,980 bytes) of heap memory
Scaling row group sizes to 73.11% for 10 writers
23/04/25 03:26:26 WARN MemoryManager: Total allocation exceeds 95.00% (981,204,980 bytes) of heap memory
Scaling row group sizes to 81.23% for 9 writers
23/04/25 03:26:26 WARN MemoryManager: Total allocation exceeds 95.00% (981,204,

,date,date_br,humidity,pressure,rain,wind,temperature,id,name,state,country,dt_processed,dt_processed_m
0,2023-04-27 18:00:00,27/04/2023 18:00:00,{'humidity': 60.9},{'pressure': 1010.1},{'precipitation': 0.7},"{'velocity': 11, 'direction': 'WSW', 'directio...",{'temperature': 23.6},3477,São Paulo,SP,BR,2023-04-25 03:16:27.198626,2023-04-01
1,2023-04-27 19:00:00,27/04/2023 19:00:00,{'humidity': 71.5},{'pressure': 1010.7},{'precipitation': 0.7},"{'velocity': 9.6, 'direction': 'SSW', 'directi...",{'temperature': 24.5},3477,São Paulo,SP,BR,2023-04-25 03:16:27.198626,2023-04-01
2,2023-04-27 20:00:00,27/04/2023 20:00:00,{'humidity': 79.4},{'pressure': 1011.1},{'precipitation': 0.7},"{'velocity': 11, 'direction': 'S', 'directiond...",{'temperature': 25.3},3477,São Paulo,SP,BR,2023-04-25 03:16:27.198626,2023-04-01
3,2023-04-27 21:00:00,27/04/2023 21:00:00,{'humidity': 86.6},{'pressure': 1011.6},{'precipitation': 0.7},"{'velocity': 14.1, 'direction': 'S', 'directio...",{'temperature': 26},3477,São Paulo,SP,BR,2023-04-25 03:16:27.198626,2023-04-01
4,2023-04-27 22:00:00,27/04/2023 22:00:00,{'humidity': 90.8},{'pressure': 1012.2},{'precipitation': 0.7},"{'velocity': 13.2, 'direction': 'S', 'directio...",{'temperature': 26},3477,São Paulo,SP,BR,2023-04-25 03:16:27.198626,2023-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,2023-04-27 07:00:00,27/04/2023 07:00:00,{'humidity': 68.8},{'pressure': 1010.8},{'precipitation': 0},"{'velocity': 10.9, 'direction': 'NW', 'directi...",{'temperature': 19.9},3477,São Paulo,SP,BR,2023-04-25 03:15:53.497902,2023-04-01
212,2023-04-27 08:00:00,27/04/2023 08:00:00,{'humidity': 70.3},{'pressure': 1010.9},{'precipitation': 0},"{'velocity': 10.9, 'direction': 'NW', 'directi...",{'temperature': 19.4},3477,São Paulo,SP,BR,2023-04-25 03:15:53.497902,2023-04-01
213,2023-04-27 09:00:00,27/04/2023 09:00:00,{'humidity': 71.6},{'pressure': 1011.3},{'precipitation': 0},"{'velocity': 11.6, 'direction': 'NW', 'directi...",{'temperature': 19.2},3477,São Paulo,SP,BR,2023-04-25 03:15:53.497902,2023-04-01
214,2023-04-27 10:00:00,27/04/2023 10:00:00,{'humidity': 71.9},{'pressure': 1012},{'precipitation': 0},"{'velocity': 13, 'direction': 'NW', 'direction...",{'temperature': 18.9},3477,São Paulo,SP,BR,2023-04-25 03:15:53.497902,2023-04-01
